In [2]:
from rdflib import Graph
from SPARQLWrapper import SPARQLWrapper, JSON, N3
from pprint import pprint
import requests

In [3]:

candidate_cells = ['Wien', 'Thessaloniki', 'Batumi', 'Limassol', 'Ganja'
, 'Capital City of Budapest', 'Kharkiv (Харків)', 'Luzern', 'Korzeniew', 'Almaty', 'Kaunas', 'Haraçinë', 'Poitiers', 'Tallinn',
 'Ljubljana', 'Երևան', 'Pristina', 'Tuzla', 'Bergen', 'Budapest főváros', 'Istanbul', 'Cwmffrwd', 'Yerevan']

def dbo_sparql_results(query_string):
    sparql = SPARQLWrapper('https://dbpedia.org/sparql')
    sparql.setQuery(query_string)
    
    try:
        sparql.setReturnFormat(JSON)
        qres = sparql.query().convert()
        return qres
    except:
        pass


def get_dbo_classes_sparql(candidate_cells):
    
    classes = list([])

    dbo_prefix = 'http://dbpedia.org/ontology/'
    for cell in candidate_cells:
        print(f'###################{cell}########################')
        query_string = f'''
        SELECT ?class
        WHERE {{ dbr:{cell} a ?class.
        }}'''
    
#         query_string = f'''
#         select distinct ?superclass 
#         where {{dbr:{cell} rdf:type ?e. 
#             ?e rdfs:subClassOf* ?superclass.
#         FILTER (strstarts(str(?superclass), '{dbo_prefix}'))}}'''
        
#         print(query_string)
    
        qres = dbo_sparql_results(query_string)
#         pprint(qres)
        try:
            for entity_class in qres['results']['bindings']:
                if dbo_prefix in entity_class[list(qres['results']['bindings'][1].keys())[0]]['value']:
                    candicate_class = entity_class[list(qres['results']['bindings'][1].keys())[0]]['value'].split(dbo_prefix)[1]
                    classes.append(candicate_class)
                    print(candicate_class)
        except:
            pass
    return set(classes)

classes = get_dbo_classes_sparql(candidate_cells)
classes

###################Wien########################
###################Thessaloniki########################
Place
Location
City
Settlement
PopulatedPlace
###################Batumi########################
Place
Location
City
Settlement
PopulatedPlace
###################Limassol########################
Place
Location
City
Settlement
PopulatedPlace
###################Ganja########################
###################Capital City of Budapest########################
###################Kharkiv (Харків)########################
###################Luzern########################
###################Korzeniew########################
Place
Location
Settlement
Village
PopulatedPlace
###################Almaty########################
Place
Location
City
Settlement
PopulatedPlace
###################Kaunas########################
Place
Location
City
Settlement
PopulatedPlace
###################Haraçinë########################
###################Poitiers########################
Place
Location
City
Settlement


{'City',
 'Location',
 'Place',
 'PopulatedPlace',
 'Settlement',
 'Town',
 'Village'}

In [6]:
def get_dbo_class_entities_sparql(candidate_classes):
    sparql = SPARQLWrapper('https://dbpedia.org/sparql')
    
    general_entities = list([])

    dbp_prefix = 'http://dbpedia.org/resource/'
    for candidate_class in candidate_classes:
        print(f'###################{candidate_class}########################')
        sparql.setQuery(f'''
        SELECT ?object
        WHERE {{ ?object a dbo:{candidate_class}. }}
        ORDER BY RAND()
        LIMIT 1000
        OFFSET 10000
        # OFFSET """ + str(i) + "0000"
        ''')
    
        try:
            sparql.setReturnFormat(JSON)
            qres = sparql.query().convert()
#             pprint(qres)
            for entity in qres['results']['bindings']:
#                 print(entity['object']['value'].split(dbp_prefix)[1])
                general_entities.append(entity['object']['value'].split(dbp_prefix)[1])
#                 if dbo_prefix in entity_class['class']['value']:
#                     candicate_class = entity_class['class']['value'].split(dbo_prefix)[1]
#                     classes.append(candicate_class)
#                     print(candicate_class)
        except:
            pass
        
    return general_entities

general_entities = get_dbo_class_entities_sparql(['AdministrativeRegion'])

###################AdministrativeRegion########################


In [9]:
# general_entities

In [5]:
general_entities

['Janowo,_Łomża_County',
 'Kampong_Lugu',
 'Cehnice',
 'Chenarestan-e_Sofla',
 'Łukowiec,_Kuyavian-Pomeranian_Voivodeship',
 'Enford',
 'Baikunthapur',
 'Wyton,_Cambridgeshire',
 'Topoľníky',
 'Němčice_(Strakonice_District)']

In [6]:
# Query DBPedia for an entity's complete classes
def query_complete_classes_of_entity(ent):
    classes = set()
    try:
        s = sparql.Service('http://dbpedia.org/sparql', "utf-8", "GET")
        statement = 'select distinct ?superclass where { <%s%s> rdf:type ?e. ' \
            '?e rdfs:subClassOf* ?superclass. FILTER(strstarts(str(?superclass), "%s"))}' \
            % (dbp_prefix, ent, dbo_prefix)
        result = s.query(statement)
        for row in result.fetchone():
            cls_uri = str(row[0])
            cls = cls_uri.split(dbo_prefix)[1]
            classes.add(cls)
    except UnicodeDecodeError:
        print('     %s: UnicodeDecodeError' % ent)
        pass
    return classes

temp = query_complete_classes_of_entity('Athens')

NameError: name 'sparql' is not defined